In [ ]:
!pip install evaluate
!pip install bert-score==0.3.13
!pip install rouge-score==0.1.2
!pip install demoji

In [ ]:
import pandas as pd
import re
import os
import pathlib
from typing import TypeVar, Dict, List, Tuple
from statistics import mean
from tqdm import tqdm
from evaluate import load
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import demoji
import string

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Πτυχιακή/Code/Experiments_Meltemi/Track_2_KPG/arg_clustering/generated_kps_4_shot.csv')
df

,Meltemi_Instruct_16shot,Representative_Docs,topic,stance
0,Οι εμβολιασμοί ρουτίνας στην παιδική ηλικία δε...,"['τα παιδιά είναι μια ευαίσθητη περίπτωση, για...",Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1
1,Ο υποχρεωτικός εμβολιασμός των παιδιών δεν πρέ...,['Τα παιδιά έχουν ένα πολύ πιο δυνατό και προσ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1
2,Τα εμβόλια μπορεί να είναι επιβλαβή για τα παιδιά,"['Ένα εμβόλιο, το οποίο δεν έχει ελεγχθεί επαρ...",Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1
3,Η υποχρεωτικότητα των εμβολιασμών παραβιάζει τ...,['Αν επιβληθεί στους γονείς να εμβολιάσουν τα ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1
4,Ο εμβολιασμός μπορεί να έχει απρόβλεπτες παρεν...,['μπορεί να δημιουργηθεί μια χημική αντίδραση ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1
5,Η προσωπική αυτονομία είναι πιο σημαντική από ...,"['Δεν είναι υποχρεωτικό, γιατί είναι μια πολύ ...",Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1
6,Οι γονείς θα πρέπει να είναι υπεύθυνοι για την...,['Οι γονείς που συμφωνούν με τη χρήση εμβολίων...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1
7,Δεν θα πρέπει να είναι υποχρεωτικό το εμβόλιο,"['δεν θα πρέπει να είναι υποχρεωτικά, καθώς έχ...",Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1
8,Το να είναι υποχρεωτικό το μάθημα σεξουαλικής ...,['Αντιβαίνει στην ελευθερία των γονέων να διαλ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1
9,Η φροντίδα των παιδιών είναι ευθύνη των γονέων,['Οι γονείς του παιδιού είναι αυτοί που πρέπει...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1


Text preprocessing for rouge compuation (stemming)

In [4]:
__cases = dict()
__cases["ΦΑΓΙΑ"] = "ΦΑ"
__cases["ΦΑΓΙΟΥ"] = "ΦΑ"
__cases["ΦΑΓΙΩΝ"] = "ΦΑ"
__cases["ΣΚΑΓΙΑ"] = "ΣΚΑ"
__cases["ΣΚΑΓΙΟΥ"] = "ΣΚΑ"
__cases["ΣΚΑΓΙΩΝ"] = "ΣΚΑ"
__cases["ΟΛΟΓΙΟΥ"] = "ΟΛΟ"
__cases["ΟΛΟΓΙΑ"] = "ΟΛΟ"
__cases["ΟΛΟΓΙΩΝ"] = "ΟΛΟ"
__cases["ΣΟΓΙΟΥ"] = "ΣΟ"
__cases["ΣΟΓΙΑ"] = "ΣΟ"
__cases["ΣΟΓΙΩΝ"] = "ΣΟ"
__cases["ΤΑΤΟΓΙΑ"] = "ΤΑΤΟ"
__cases["ΤΑΤΟΓΙΟΥ"] = "ΤΑΤΟ"
__cases["ΤΑΤΟΓΙΩΝ"] = "ΤΑΤΟ"
__cases["ΚΡΕΑΣ"] = "ΚΡΕ"
__cases["ΚΡΕΑΤΟΣ"] = "ΚΡΕ"
__cases["ΚΡΕΑΤΑ"] = "ΚΡΕ"
__cases["ΚΡΕΑΤΩΝ"] = "ΚΡΕ"
__cases["ΠΕΡΑΣ"] = "ΠΕΡ"
__cases["ΠΕΡΑΤΟΣ"] = "ΠΕΡ"
__cases["ΠΕΡΑΤΑ"] = "ΠΕΡ"
__cases["ΠΕΡΑΤΩΝ"] = "ΠΕΡ"
__cases["ΤΕΡΑΣ"] = "ΤΕΡ"
__cases["ΤΕΡΑΤΟΣ"] = "ΤΕΡ"
__cases["ΤΕΡΑΤΑ"] = "ΤΕΡ"
__cases["ΤΕΡΑΤΩΝ"] = "ΤΕΡ"
__cases["ΦΩΣ"] = "ΦΩ"
__cases["ΦΩΤΟΣ"] = "ΦΩ"
__cases["ΦΩΤΑ"] = "ΦΩ"
__cases["ΦΩΤΩΝ"] = "ΦΩ"
__cases["ΚΑΘΕΣΤΩΣ"] = "ΚΑΘΕΣΤ"
__cases["ΚΑΘΕΣΤΩΤΟΣ"] = "ΚΑΘΕΣΤ"
__cases["ΚΑΘΕΣΤΩΤΑ"] = "ΚΑΘΕΣΤ"
__cases["ΚΑΘΕΣΤΩΤΩΝ"] = "ΚΑΘΕΣΤ"
__cases["ΓΕΓΟΝΟΣ"] = "ΓΕΓΟΝ"
__cases["ΓΕΓΟΝΟΤΟΣ"] = "ΓΕΓΟΝ"
__cases["ΓΕΓΟΝΟΤΑ"] = "ΓΕΓΟΝ"
__cases["ΓΕΓΟΝΟΤΩΝ"] = "ΓΕΓΟΝ"
__vowels = "[ΑΕΗΙΟΥΩ]"
__refinedVowels = "[ΑΕΗΙΟΩ]"


def stemWord(w: str, exceptions: dict = None):
    stem = None
    suffix = None
    test1 = True

    if exceptions is not None and w in exceptions.keys():
        return exceptions[w]

    if len(w) < 4:
        return w

    pattern = None
    pattern2 = None
    pattern3 = None
    pattern4 = None

    # Step1
    pattern = re.compile(
        r"(.*)(ΦΑΓΙΑ|ΦΑΓΙΟΥ|ΦΑΓΙΩΝ|ΣΚΑΓΙΑ|ΣΚΑΓΙΟΥ|ΣΚΑΓΙΩΝ|ΟΛΟΓΙΟΥ|ΟΛΟΓΙΑ|ΟΛΟΓΙΩΝ|ΣΟΓΙΟΥ|ΣΟΓΙΑ|ΣΟΓΙΩΝ|ΤΑΤΟΓΙΑ|ΤΑΤΟΓΙΟΥ|ΤΑΤΟΓΙΩΝ|ΚΡΕΑΣ|ΚΡΕΑΤΟΣ|ΚΡΕΑΤΑ|ΚΡΕΑΤΩΝ|ΠΕΡΑΣ|ΠΕΡΑΤΟΣ|ΠΕΡΑΤΑ|ΠΕΡΑΤΩΝ|ΤΕΡΑΣ|ΤΕΡΑΤΟΣ|ΤΕΡΑΤΑ|ΤΕΡΑΤΩΝ|ΦΩΣ|ΦΩΤΟΣ|ΦΩΤΑ|ΦΩΤΩΝ|ΚΑΘΕΣΤΩΣ|ΚΑΘΕΣΤΩΤΟΣ|ΚΑΘΕΣΤΩΤΑ|ΚΑΘΕΣΤΩΤΩΝ|ΓΕΓΟΝΟΣ|ΓΕΓΟΝΟΤΟΣ|ΓΕΓΟΝΟΤΑ|ΓΕΓΟΝΟΤΩΝ)$")

    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        suffix = fp[1]
        w = stem + __cases[suffix]
        test1 = False

    # Step 2a
    pattern = re.compile(r"^(.+?)(ΑΔΕΣ|ΑΔΩΝ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        reg1 = re.compile(r"(ΟΚ|ΜΑΜ|ΜΑΝ|ΜΠΑΜΠ|ΠΑΤΕΡ|ΓΙΑΓΙ|ΝΤΑΝΤ|ΚΥΡ|ΘΕΙ|ΠΕΘΕΡ)$")

        if not reg1.match(w):
            w = w + "ΑΔ"

    # Step 2b
    pattern2 = re.compile(r"^(.+?)(ΕΔΕΣ|ΕΔΩΝ)$")
    if pattern2.match(w):
        fp = pattern2.match(w).groups()
        stem = fp[0]
        w = stem
        except2 = re.compile(r"(ΟΠ|ΙΠ|ΕΜΠ|ΥΠ|ΓΗΠ|ΔΑΠ|ΚΡΑΣΠ|ΜΙΛ)$")
        if except2.match(w):
            w = w + "ΕΔ"

    # Step 2c
    pattern3 = re.compile(r"^(.+?)(ΟΥΔΕΣ|ΟΥΔΩΝ)$")
    if pattern3.match(w):
        fp = pattern3.match(w).groups()
        stem = fp[0]
        w = stem
        except3 = re.compile(r"(ΑΡΚ|ΚΑΛΙΑΚ|ΠΕΤΑΛ|ΛΙΧ|ΠΛΕΞ|ΣΚ|Σ|ΦΛ|ΦΡ|ΒΕΛ|ΛΟΥΛ|ΧΝ|ΣΠ|ΤΡΑΓ|ΦΕ)$")
        if except3.match(w):
            w = w + "ΟΥΔ"

    # Step 2d
    pattern4 = re.compile("^(.+?)(ΕΩΣ|ΕΩΝ)$")
    if pattern4.match(w):
        fp = pattern4.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except4 = re.compile(r"^(Θ|Δ|ΕΛ|ΓΑΛ|Ν|Π|ΙΔ|ΠΑΡ)$")
        if except4.match(w):
            w = w + "Ε"

    # Step 3
    pattern = re.compile(r"^(.+?)(ΙΑ|ΙΟΥ|ΙΩΝ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        pattern2 = re.compile(__vowels + "$")
        test1 = False
        if pattern2.match(w):
            w = stem + "Ι"

    # Step 4
    pattern = re.compile(r"^(.+?)(ΙΚΑ|ΙΚΟ|ΙΚΟΥ|ΙΚΩΝ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        pattern2 = re.compile(__vowels + "$")
        except5 = re.compile(
            r"^(ΑΛ|ΑΔ|ΕΝΔ|ΑΜΑΝ|ΑΜΜΟΧΑΛ|ΗΘ|ΑΝΗΘ|ΑΝΤΙΔ|ΦΥΣ|ΒΡΩΜ|ΓΕΡ|ΕΞΩΔ|ΚΑΛΠ|ΚΑΛΛΙΝ|ΚΑΤΑΔ|ΜΟΥΛ|ΜΠΑΝ|ΜΠΑΓΙΑΤ|ΜΠΟΛ|ΜΠΟΣ|ΝΙΤ|ΞΙΚ|ΣΥΝΟΜΗΛ|ΠΕΤΣ|ΠΙΤΣ|ΠΙΚΑΝΤ|ΠΛΙΑΤΣ|ΠΟΣΤΕΛΝ|ΠΡΩΤΟΔ|ΣΕΡΤ|ΣΥΝΑΔ|ΤΣΑΜ|ΥΠΟΔ|ΦΙΛΟΝ|ΦΥΛΟΔ|ΧΑΣ)$")
        if except5.match(w) or pattern2.match(w):
            w = w + "ΙΚ"

    # step 5a
    pattern = re.compile(r"^(.+?)(ΑΜΕ)$")
    pattern2 = re.compile(r"^(.+?)(ΑΓΑΜΕ|ΗΣΑΜΕ|ΟΥΣΑΜΕ|ΗΚΑΜΕ|ΗΘΗΚΑΜΕ)$")
    if w == "ΑΓΑΜΕ":
        w = "ΑΓΑΜ"

    if pattern2.match(w):
        fp = pattern2.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False

    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except6 = re.compile(r"^(ΑΝΑΠ|ΑΠΟΘ|ΑΠΟΚ|ΑΠΟΣΤ|ΒΟΥΒ|ΞΕΘ|ΟΥΛ|ΠΕΘ|ΠΙΚΡ|ΠΟΤ|ΣΙΧ|Χ)$")
        if except6.match(w):
            w = w + "ΑΜ"

    # Step 5b
    pattern2 = re.compile(r"^(.+?)(ΑΝΕ)$")
    pattern3 = re.compile(r"^(.+?)(ΑΓΑΝΕ|ΗΣΑΝΕ|ΟΥΣΑΝΕ|ΙΟΝΤΑΝΕ|ΙΟΤΑΝΕ|ΙΟΥΝΤΑΝΕ|ΟΝΤΑΝΕ|ΟΤΑΝΕ|ΟΥΝΤΑΝΕ|ΗΚΑΝΕ|ΗΘΗΚΑΝΕ)$")
    if pattern3.match(w):
        fp = pattern3.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        pattern3 = re.compile(r"^(ΤΡ|ΤΣ)$")
        if pattern3.match(w):
            w = w + "ΑΓΑΝ"

    if pattern2.match(w):
        fp = pattern2.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        pattern2 = re.compile(__refinedVowels + "$")
        except7 = re.compile(
            r"^(ΒΕΤΕΡ|ΒΟΥΛΚ|ΒΡΑΧΜ|Γ|ΔΡΑΔΟΥΜ|Θ|ΚΑΛΠΟΥΖ|ΚΑΣΤΕΛ|ΚΟΡΜΟΡ|ΛΑΟΠΛ|ΜΩΑΜΕΘ|Μ|ΜΟΥΣΟΥΛΜ|Ν|ΟΥΛ|Π|ΠΕΛΕΚ|ΠΛ|ΠΟΛΙΣ|ΠΟΡΤΟΛ|ΣΑΡΑΚΑΤΣ|ΣΟΥΛΤ|ΤΣΑΡΛΑΤ|ΟΡΦ|ΤΣΙΓΓ|ΤΣΟΠ|ΦΩΤΟΣΤΕΦ|Χ|ΨΥΧΟΠΛ|ΑΓ|ΟΡΦ|ΓΑΛ|ΓΕΡ|ΔΕΚ|ΔΙΠΛ|ΑΜΕΡΙΚΑΝ|ΟΥΡ|ΠΙΘ|ΠΟΥΡΙΤ|Σ|ΖΩΝΤ|ΙΚ|ΚΑΣΤ|ΚΟΠ|ΛΙΧ|ΛΟΥΘΗΡ|ΜΑΙΝΤ|ΜΕΛ|ΣΙΓ|ΣΠ|ΣΤΕΓ|ΤΡΑΓ|ΤΣΑΓ|Φ|ΕΡ|ΑΔΑΠ|ΑΘΙΓΓ|ΑΜΗΧ|ΑΝΙΚ|ΑΝΟΡΓ|ΑΠΗΓ|ΑΠΙΘ|ΑΤΣΙΓΓ|ΒΑΣ|ΒΑΣΚ|ΒΑΘΥΓΑΛ|ΒΙΟΜΗΧ|ΒΡΑΧΥΚ|ΔΙΑΤ|ΔΙΑΦ|ΕΝΟΡΓ|ΘΥΣ|ΚΑΠΝΟΒΙΟΜΗΧ|ΚΑΤΑΓΑΛ|ΚΛΙΒ|ΚΟΙΛΑΡΦ|ΛΙΒ|ΜΕΓΛΟΒΙΟΜΗΧ|ΜΙΚΡΟΒΙΟΜΗΧ|ΝΤΑΒ|ΞΗΡΟΚΛΙΒ|ΟΛΙΓΟΔΑΜ|ΟΛΟΓΑΛ|ΠΕΝΤΑΡΦ|ΠΕΡΗΦ|ΠΕΡΙΤΡ|ΠΛΑΤ|ΠΟΛΥΔΑΠ|ΠΟΛΥΜΗΧ|ΣΤΕΦ|ΤΑΒ|ΤΕΤ|ΥΠΕΡΗΦ|ΥΠΟΚΟΠ|ΧΑΜΗΛΟΔΑΠ|ΨΗΛΟΤΑΒ)$")
        if (pattern2.match(w)) or (except7.match(w)):
            w = w + "ΑΝ"

    # //Step 5c
    pattern3 = re.compile(r"^(.+?)(ΕΤΕ)$")
    pattern4 = re.compile(r"^(.+?)(ΗΣΕΤΕ)$")
    if pattern4.match(w):
        fp = pattern4.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False

    if pattern3.match(w):
        fp = pattern3.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        pattern3 = re.compile(__refinedVowels + "$")
        except8 = re.compile(
            r"(ΟΔ|ΑΙΡ|ΦΟΡ|ΤΑΘ|ΔΙΑΘ|ΣΧ|ΕΝΔ|ΕΥΡ|ΤΙΘ|ΥΠΕΡΘ|ΡΑΘ|ΕΝΘ|ΡΟΘ|ΣΘ|ΠΥΡ|ΑΙΝ|ΣΥΝΔ|ΣΥΝ|ΣΥΝΘ|ΧΩΡ|ΠΟΝ|ΒΡ|ΚΑΘ|ΕΥΘ|ΕΚΘ|ΝΕΤ|ΡΟΝ|ΑΡΚ|ΒΑΡ|ΒΟΛ|ΩΦΕΛ)$")
        except9 = re.compile(
            r"^(ΑΒΑΡ|ΒΕΝ|ΕΝΑΡ|ΑΒΡ|ΑΔ|ΑΘ|ΑΝ|ΑΠΛ|ΒΑΡΟΝ|ΝΤΡ|ΣΚ|ΚΟΠ|ΜΠΟΡ|ΝΙΦ|ΠΑΓ|ΠΑΡΑΚΑΛ|ΣΕΡΠ|ΣΚΕΛ|ΣΥΡΦ|ΤΟΚ|Υ|Δ|ΕΜ|ΘΑΡΡ|Θ)$")
        if (pattern3.match(w)) or (except8.match(w)) or (except9.match(w)):
            w = w + "ΕΤ"

    # Step 5d
    pattern = re.compile(r"^(.+?)(ΟΝΤΑΣ|ΩΝΤΑΣ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except10 = re.compile(r"^(ΑΡΧ)$")
        except11 = re.compile(r"(ΚΡΕ)$")
        if except10.match(w):
            w = w + "ΟΝΤ"
        if except11.match(w):
            w = w + "ΩΝΤ"

    # Step 5e
    pattern = re.compile(r"^(.+?)(ΟΜΑΣΤΕ|ΙΟΜΑΣΤΕ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except11 = re.compile("^(ΟΝ)$")
        if except11.match(w):
            w = w + "ΟΜΑΣΤ"

    # Step 5f
    pattern = re.compile(r"^(.+?)(ΕΣΤΕ)$")
    pattern2 = re.compile(r"^(.+?)(ΙΕΣΤΕ)$")
    if pattern2.match(w):
        fp = pattern2.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        pattern2 = re.compile(r"^(Π|ΑΠ|ΣΥΜΠ|ΑΣΥΜΠ|ΑΚΑΤΑΠ|ΑΜΕΤΑΜΦ)$")
        if pattern2.match(w):
            w = w + "ΙΕΣΤ"

    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except12 = re.compile(r"^(ΑΛ|ΑΡ|ΕΚΤΕΛ|Ζ|Μ|Ξ|ΠΑΡΑΚΑΛ|ΑΡ|ΠΡΟ|ΝΙΣ)$")
        if except12.match(w):
            w = w + "ΕΣΤ"

    # Step 5g
    pattern = re.compile(r"^(.+?)(ΗΚΑ|ΗΚΕΣ|ΗΚΕ)$")
    pattern2 = re.compile(r"^(.+?)(ΗΘΗΚΑ|ΗΘΗΚΕΣ|ΗΘΗΚΕ)$")
    if pattern2.match(w):
        fp = pattern2.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False

    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except13 = re.compile(r"(ΣΚΩΛ|ΣΚΟΥΛ|ΝΑΡΘ|ΣΦ|ΟΘ|ΠΙΘ)$")
        except14 = re.compile(r"^(ΔΙΑΘ|Θ|ΠΑΡΑΚΑΤΑΘ|ΠΡΟΣΘ|ΣΥΝΘ|)$")
        if (except13.match(w)) or (except14.match(w)):
            w = w + "ΗΚ"

    # Step 5h
    pattern = re.compile(r"^(.+?)(ΟΥΣΑ|ΟΥΣΕΣ|ΟΥΣΕ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except15 = re.compile(
            r"^(ΦΑΡΜΑΚ|ΧΑΔ|ΑΓΚ|ΑΝΑΡΡ|ΒΡΟΜ|ΕΚΛΙΠ|ΛΑΜΠΙΔ|ΛΕΧ|Μ|ΠΑΤ|Ρ|Λ|ΜΕΔ|ΜΕΣΑΖ|ΥΠΟΤΕΙΝ|ΑΜ|ΑΙΘ|ΑΝΗΚ|ΔΕΣΠΟΖ|ΕΝΔΙΑΦΕΡ|ΔΕ|ΔΕΥΤΕΡΕΥ|ΚΑΘΑΡΕΥ|ΠΛΕ|ΤΣΑ)$")
        except16 = re.compile(r"(ΠΟΔΑΡ|ΒΛΕΠ|ΠΑΝΤΑΧ|ΦΡΥΔ|ΜΑΝΤΙΛ|ΜΑΛΛ|ΚΥΜΑΤ|ΛΑΧ|ΛΗΓ|ΦΑΓ|ΟΜ|ΠΡΩΤ)$")
        if (except15.match(w)) or (except16.match(w)):
            w = w + "ΟΥΣ"

    # Step 5i
    pattern = re.compile(r"^(.+?)(ΑΓΑ|ΑΓΕΣ|ΑΓΕ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except17 = re.compile(r"^(ΨΟΦ|ΝΑΥΛΟΧ)$")
        except20 = re.compile(r"(ΚΟΛΛ)$")
        except18 = re.compile(
            r"^(ΑΒΑΣΤ|ΠΟΛΥΦ|ΑΔΗΦ|ΠΑΜΦ|Ρ|ΑΣΠ|ΑΦ|ΑΜΑΛ|ΑΜΑΛΛΙ|ΑΝΥΣΤ|ΑΠΕΡ|ΑΣΠΑΡ|ΑΧΑΡ|ΔΕΡΒΕΝ|ΔΡΟΣΟΠ|ΞΕΦ|ΝΕΟΠ|ΝΟΜΟΤ|ΟΛΟΠ|ΟΜΟΤ|ΠΡΟΣΤ|ΠΡΟΣΩΠΟΠ|ΣΥΜΠ|ΣΥΝΤ|Τ|ΥΠΟΤ|ΧΑΡ|ΑΕΙΠ|ΑΙΜΟΣΤ|ΑΝΥΠ|ΑΠΟΤ|ΑΡΤΙΠ|ΔΙΑΤ|ΕΝ|ΕΠΙΤ|ΚΡΟΚΑΛΟΠ|ΣΙΔΗΡΟΠ|Λ|ΝΑΥ|ΟΥΛΑΜ|ΟΥΡ|Π|ΤΡ|Μ)$")
        except19 = re.compile(r"(ΟΦ|ΠΕΛ|ΧΟΡΤ|ΛΛ|ΣΦ|ΡΠ|ΦΡ|ΠΡ|ΛΟΧ|ΣΜΗΝ)$")
        if (except18.match(w) and except19.match(w)) and not ((except17.match(w)) or (except20.match(w))):
            w = w + "ΑΓ"

    # Step 5j
    pattern = re.compile("^(.+?)(ΗΣΕ|ΗΣΟΥ|ΗΣΑ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except21 = re.compile(r"^(Ν|ΧΕΡΣΟΝ|ΔΩΔΕΚΑΝ|ΕΡΗΜΟΝ|ΜΕΓΑΛΟΝ|ΕΠΤΑΝ)$")
        if except21.match(w):
            w = w + "ΗΣ"

    # Step 5k
    pattern = re.compile(r"^(.+?)(ΗΣΤΕ)$")

    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except22 = re.compile(r"^(ΑΣΒ|ΣΒ|ΑΧΡ|ΧΡ|ΑΠΛ|ΑΕΙΜΝ|ΔΥΣΧΡ|ΕΥΧΡ|ΚΟΙΝΟΧΡ|ΠΑΛΙΜΨ)$")
        if except22.match(w):
            w = w + "ΗΣΤ"

    # Step 5l
    pattern = re.compile("^(.+?)(ΟΥΝΕ|ΗΣΟΥΝΕ|ΗΘΟΥΝΕ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except23 = re.compile("^(Ν|Ρ|ΣΠΙ|ΣΤΡΑΒΟΜΟΥΤΣ|ΚΑΚΟΜΟΥΤΣ|ΕΞΩΝ)$")
        if except23.match(w):
            w = w + "ΟΥΝ"

    # Step 5l
    pattern = re.compile(r"^(.+?)(ΟΥΜΕ|ΗΣΟΥΜΕ|ΗΘΟΥΜΕ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except24 = re.compile(r"^(ΠΑΡΑΣΟΥΣ|Φ|Χ|ΩΡΙΟΠΛ|ΑΖ|ΑΛΛΟΣΟΥΣ|ΑΣΟΥΣ)$")
        if except24.match(w):
            w = w + "ΟΥΜ"

    # Step 6
    pattern = re.compile(r"^(.+?)(ΜΑΤΑ|ΜΑΤΩΝ|ΜΑΤΟΣ)$")
    pattern2 = re.compile(
        r"^(.+?)(Α|ΑΓΑΤΕ|ΑΓΑΝ|ΑΕΙ|ΑΜΑΙ|ΑΝ|ΑΣ|ΑΣΑΙ|ΑΤΑΙ|ΑΩ|Ε|ΕΙ|ΕΙΣ|ΕΙΤΕ|ΕΣΑΙ|ΕΣ|ΕΤΑΙ|Ι|ΙΕΜΑΙ|ΙΕΜΑΣΤΕ|ΙΕΤΑΙ|ΙΕΣΑΙ|ΙΕΣΑΣΤΕ|ΙΟΜΑΣΤΑΝ|ΙΟΜΟΥΝ|ΙΟΜΟΥΝΑ|ΙΟΝΤΑΝ|ΙΟΝΤΟΥΣΑΝ|ΙΟΣΑΣΤΑΝ|ΙΟΣΑΣΤΕ|ΙΟΣΟΥΝ|ΙΟΣΟΥΝΑ|ΙΟΤΑΝ|ΙΟΥΜΑ|ΙΟΥΜΑΣΤΕ|ΙΟΥΝΤΑΙ|ΙΟΥΝΤΑΝ|Η|ΗΔΕΣ|ΗΔΩΝ|ΗΘΕΙ|ΗΘΕΙΣ|ΗΘΕΙΤΕ|ΗΘΗΚΑΤΕ|ΗΘΗΚΑΝ|ΗΘΟΥΝ|ΗΘΩ|ΗΚΑΤΕ|ΗΚΑΝ|ΗΣ|ΗΣΑΝ|ΗΣΑΤΕ|ΗΣΕΙ|ΗΣΕΣ|ΗΣΟΥΝ|ΗΣΩ|Ο|ΟΙ|ΟΜΑΙ|ΟΜΑΣΤΑΝ|ΟΜΟΥΝ|ΟΜΟΥΝΑ|ΟΝΤΑΙ|ΟΝΤΑΝ|ΟΝΤΟΥΣΑΝ|ΟΣ|ΟΣΑΣΤΑΝ|ΟΣΑΣΤΕ|ΟΣΟΥΝ|ΟΣΟΥΝΑ|ΟΤΑΝ|ΟΥ|ΟΥΜΑΙ|ΟΥΜΑΣΤΕ|ΟΥΝ|ΟΥΝΤΑΙ|ΟΥΝΤΑΝ|ΟΥΣ|ΟΥΣΑΝ|ΟΥΣΑΤΕ|Υ|ΥΣ|Ω|ΩΝ)$")

    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem + "ΜΑ"

    if pattern2.match(w) and test1:
        fp = pattern2.match(w).groups()
        stem = fp[0]
        w = stem

    # Step 7 (ΠΑΡΑΘΕΤΙΚΑ)
    pattern = re.compile(r"^(.+?)(ΕΣΤΕΡ|ΕΣΤΑΤ|ΟΤΕΡ|ΟΤΑΤ|ΥΤΕΡ|ΥΤΑΤ|ΩΤΕΡ|ΩΤΑΤ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem

    return w

# Notes:
# (1) Greek word to get stemmed should be in capital letters and without accents. E.g. ΨΩΜΙ and not Ψωμί
# (2) Any word of length 1 - 3 will not get stemmed.
# (3) The exceptions argument is a dictionary where a key is a word and a value is a custom stem of your liking

In [5]:
def replaceMultiple(main, replacements, new):
  for elem in replacements:
    if elem in main:
      main = main.replace(elem, new)
  return main

def normalize(x):
  x = replaceMultiple(x, ['ά', 'ὰ', 'ἀ','ἁ','ἂ','ἃ','ἄ','ἅ','ἆ','ἇ','ᾰ','ᾱ','ᾲ','ᾳ','ᾴ','ᾶ','ᾷ'], 'α')
  x = replaceMultiple(x, ['έ','ὲ','ἐ','ἑ','ἒ','ἓ','ἔ','ἕ'], 'ε')
  x = replaceMultiple(x, ['ή','ὴ','ἠ','ἡ','ἢ','ἣ','ἤ','ἥ','ἦ','ἧ','ῂ','ῃ','ῄ','ῆ','ῇ'], 'η')
  x = replaceMultiple(x, ['ί','ὶ','ἰ','ἱ','ἲ','ἳ','ἴ','ἵ','ἶ','ἷ','ῐ','ῑ','ῒ','ΐ','ῖ','ῗ'], 'ι')
  x = replaceMultiple(x, ['ὸ','ό','ὀ','ὁ','ὂ','ὃ','ὄ','ὅ'], 'ο')
  x = replaceMultiple(x, ['ύ','ὺ','ΰ', 'ϋ','ὐ','ὑ','ὒ','ὓ','ὔ','ὕ','ὖ','ὗ','ῠ','ῡ','ῢ','ΰ','ῦ'], 'υ')
  x = replaceMultiple(x, ['ώ','ὼ','ῶ','ῲ','ῳ','ῴ','ῶ','ῷ','ὠ','ὡ','ὢ','ὣ','ὤ','ὥ','ὦ','ὧ'], 'ω')
  return x

def remove_emojis(text):
    return demoji.replace(text, '')

def sep_punc(x):
    punc = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~؛،؟؛.»«”'
    out = []
    for char in x:
        if char in punc:
            out.append(' ' + char + ' ')
        else:
            out.append(char)
    return ''.join(out)

def preprocess_text(text):
    # Remove retweets
    text = re.sub(r'^RT[\s]+', '', text, flags=re.IGNORECASE)
    # Remove usernames @
    text = re.sub(r'@[^\s]+', '', text)
    # Remove URLs
    text = re.sub(r'https\S+', '', text)
    url_words = ['url', 'URL', 'html', 'HTML', 'http', 'HTTP']
    for u in url_words:
        text = re.sub(u, '', text)
    text = text.lower()
    text=normalize(text)
     # Separate punctuation characters
    text = sep_punc(text)
    # Remove punctuation characters
    text = re.sub(f"[{''.join(re.escape(c) for c in string.punctuation if c != '_') }]", ' ', text)
    # Remove Latin characters
    text = re.sub(r'[a-zA-Z]+', '', text)
    # Remove numbers
    text = re.sub(r'\b\d+\b', '', text)
  # Remove words with character length equal to or less than 1
    words = text.split()
    filtered_words = [word for word in words if len(word) > 1]
    text = ' '.join(filtered_words)
    text = remove_emojis(text)
    text = str(text).upper().strip()
    tokenized_tweet = word_tokenize(text)
    tokenized_tweet = [stemWord(i) for i in tokenized_tweet]
    return ' '.join(tokenized_tweet)

In [6]:
def evaluate_and_store_with_preprocessing(final_kp_df: pd.DataFrame) -> pd.DataFrame:
    """
    This function computes the ROUGE-1 and BERTScore for each preprocessed keypoint and keypoint_given
    in the DataFrame. The scores are stored as new columns in the DataFrame.

    Parameters
    -----------
    final_kp_df: DataFrame containing 'keypoint' (predictions) and 'keypoint_given' (references) columns (pd.DataFrame).

    Returns
    --------
    final_kp_df: Updated DataFrame with 'rouge1' and 'bertscore_f1' columns added (pd.DataFrame).
    """
    # Load the evaluation metrics.
    bertscore = load('bertscore')
    rouge = load('rouge')

    # Initialize lists to store the scores for each row.
    rouge1_scores = []
    bertscore_f1_scores = []

    # Iterate through each row in the DataFrame and compute the metrics.
    for index, row in tqdm(final_kp_df.iterrows(), total=len(final_kp_df), desc='Computing metrics for each row...'):
        # Preprocess the prediction and reference text
        prediction = preprocess_text(row['Meltemi_Instruct_16shot'])
        reference = preprocess_text(row['topic'])

        # Compute ROUGE-1 score for the current row.
        rouge_scores = rouge.compute(predictions=[prediction], references=[reference], tokenizer=lambda x: x.split())
        rouge1 = rouge_scores['rouge1']

        # Compute BERTScore F1 score for the current row.
        bertscore_scores = bertscore.compute(predictions=[prediction], references=[reference], lang='el', device='cpu')
        bert_f1 = bertscore_scores['f1'][0]  # Extract the F1 score (first element) [0]

        # Append the scores to the lists.
        rouge1_scores.append(rouge1)
        bertscore_f1_scores.append(bert_f1)

    # Add the computed scores as new columns to the DataFrame.
    final_kp_df['rouge1_score'] = rouge1_scores
    final_kp_df['f1_bertscore'] = bertscore_f1_scores

    return final_kp_df

In [7]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
final_kp_df = evaluate_and_store_with_preprocessing(df)


In [9]:
final_kp_df

,Meltemi_Instruct_16shot,Representative_Docs,topic,stance,rouge1_score,f1_bertscore
0,Οι εμβολιασμοί ρουτίνας στην παιδική ηλικία δε...,"['τα παιδιά είναι μια ευαίσθητη περίπτωση, για...",Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.800000,0.953979
1,Ο υποχρεωτικός εμβολιασμός των παιδιών δεν πρέ...,['Τα παιδιά έχουν ένα πολύ πιο δυνατό και προσ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.555556,0.921875
2,Τα εμβόλια μπορεί να είναι επιβλαβή για τα παιδιά,"['Ένα εμβόλιο, το οποίο δεν έχει ελεγχθεί επαρ...",Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.222222,0.844562
3,Η υποχρεωτικότητα των εμβολιασμών παραβιάζει τ...,['Αν επιβληθεί στους γονείς να εμβολιάσουν τα ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.117647,0.858979
4,Ο εμβολιασμός μπορεί να έχει απρόβλεπτες παρεν...,['μπορεί να δημιουργηθεί μια χημική αντίδραση ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.266667,0.854395
5,Η προσωπική αυτονομία είναι πιο σημαντική από ...,"['Δεν είναι υποχρεωτικό, γιατί είναι μια πολύ ...",Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.111111,0.827848
6,Οι γονείς θα πρέπει να είναι υπεύθυνοι για την...,['Οι γονείς που συμφωνούν με τη χρήση εμβολίων...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.416667,0.868019
7,Δεν θα πρέπει να είναι υποχρεωτικό το εμβόλιο,"['δεν θα πρέπει να είναι υποχρεωτικά, καθώς έχ...",Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.470588,0.884127
8,Το να είναι υποχρεωτικό το μάθημα σεξουαλικής ...,['Αντιβαίνει στην ελευθερία των γονέων να διαλ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.260870,0.853736
9,Η φροντίδα των παιδιών είναι ευθύνη των γονέων,['Οι γονείς του παιδιού είναι αυτοί που πρέπει...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.125000,0.838840


In [10]:
df_filtered= final_kp_df[final_kp_df['f1_bertscore'] < 0.90]

In [11]:
df_filtered

,Meltemi_Instruct_16shot,Representative_Docs,topic,stance,rouge1_score,f1_bertscore
2,Τα εμβόλια μπορεί να είναι επιβλαβή για τα παιδιά,"['Ένα εμβόλιο, το οποίο δεν έχει ελεγχθεί επαρ...",Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.222222,0.844562
3,Η υποχρεωτικότητα των εμβολιασμών παραβιάζει τ...,['Αν επιβληθεί στους γονείς να εμβολιάσουν τα ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.117647,0.858979
4,Ο εμβολιασμός μπορεί να έχει απρόβλεπτες παρεν...,['μπορεί να δημιουργηθεί μια χημική αντίδραση ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.266667,0.854395
5,Η προσωπική αυτονομία είναι πιο σημαντική από ...,"['Δεν είναι υποχρεωτικό, γιατί είναι μια πολύ ...",Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.111111,0.827848
6,Οι γονείς θα πρέπει να είναι υπεύθυνοι για την...,['Οι γονείς που συμφωνούν με τη χρήση εμβολίων...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.416667,0.868019
7,Δεν θα πρέπει να είναι υποχρεωτικό το εμβόλιο,"['δεν θα πρέπει να είναι υποχρεωτικά, καθώς έχ...",Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.470588,0.884127
8,Το να είναι υποχρεωτικό το μάθημα σεξουαλικής ...,['Αντιβαίνει στην ελευθερία των γονέων να διαλ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.260870,0.853736
9,Η φροντίδα των παιδιών είναι ευθύνη των γονέων,['Οι γονείς του παιδιού είναι αυτοί που πρέπει...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.125000,0.838840
10,Τα εμβόλια θα πρέπει να είναι υποχρεωτικά για ...,['Τα εμβόλια θα πρέπει να αποτελούν υποχρεωτικ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,1,0.380952,0.882578
11,Τα παιδικά εμβόλια θα πρέπει να είναι υποχρεωτ...,['τα παιδικά εμβόλια θα πρέπει να είναι υποχρε...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,1,0.347826,0.873110


In [12]:
values_to_remove = [
    "Δεν θα πρέπει να είναι υποχρεωτικό το εμβόλιο",
    "Τα εμβόλια θα πρέπει να είναι υποχρεωτικά για όλα τα παιδιά",
]

# Filter the dataframe to exclude rows with specified values
df_filtered = df_filtered[~df_filtered["Meltemi_Instruct_4shot"].isin(values_to_remove)]

df_filtered

,Meltemi_Instruct_16shot,Representative_Docs,topic,stance,rouge1_score,f1_bertscore
2,Τα εμβόλια μπορεί να είναι επιβλαβή για τα παιδιά,"['Ένα εμβόλιο, το οποίο δεν έχει ελεγχθεί επαρ...",Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.222222,0.844562
3,Η υποχρεωτικότητα των εμβολιασμών παραβιάζει τ...,['Αν επιβληθεί στους γονείς να εμβολιάσουν τα ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.117647,0.858979
4,Ο εμβολιασμός μπορεί να έχει απρόβλεπτες παρεν...,['μπορεί να δημιουργηθεί μια χημική αντίδραση ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.266667,0.854395
5,Η προσωπική αυτονομία είναι πιο σημαντική από ...,"['Δεν είναι υποχρεωτικό, γιατί είναι μια πολύ ...",Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.111111,0.827848
6,Οι γονείς θα πρέπει να είναι υπεύθυνοι για την...,['Οι γονείς που συμφωνούν με τη χρήση εμβολίων...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.416667,0.868019
8,Το να είναι υποχρεωτικό το μάθημα σεξουαλικής ...,['Αντιβαίνει στην ελευθερία των γονέων να διαλ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.260870,0.853736
9,Η φροντίδα των παιδιών είναι ευθύνη των γονέων,['Οι γονείς του παιδιού είναι αυτοί που πρέπει...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,0.125000,0.838840
10,Τα εμβόλια θα πρέπει να είναι υποχρεωτικά για ...,['Τα εμβόλια θα πρέπει να αποτελούν υποχρεωτικ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,1,0.380952,0.882578
11,Τα παιδικά εμβόλια θα πρέπει να είναι υποχρεωτ...,['τα παιδικά εμβόλια θα πρέπει να είναι υποχρε...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,1,0.347826,0.873110
12,Τα παιδικά εμβόλια ρουτίνας θα πρέπει να είναι...,['Τα αλάνθαστα εμβόλια ρουτίνας είναι υποχρεωτ...,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,1,0.416667,0.894479


In [13]:
len(df_filtered)

42

In [14]:
from itertools import combinations

def filter_keypoints_with_logging(final_kp_df: pd.DataFrame, bertscore_threshold: float = 0.9) -> (pd.DataFrame, pd.DataFrame):
    """
    Filters keypoints within each topic and stance group to ensure diversity and logs dropped keypoints.

    Parameters
    -----------
    final_kp_df: pd.DataFrame
        DataFrame containing columns: 'topic', 'stance', and 'keypoint', ranked by importance.

    bertscore_threshold: float, optional (default=0.9)
        Threshold for BERTScore above which two keypoints are considered too similar.

    Returns
    --------
    filtered_df: pd.DataFrame
        DataFrame with redundant keypoints removed, retaining only diverse keypoints for each topic and stance group.

    dropped_keypoints_df: pd.DataFrame
        DataFrame logging the dropped keypoints and their similarity scores.
    """
    # Load the BERTScore evaluation metric
    bertscore = load('bertscore')

    # Create empty lists to store the filtered rows and dropped keypoints
    filtered_rows = []
    dropped_keypoints = []

    # Group the DataFrame by topic and stance
    grouped = final_kp_df.groupby(['topic', 'stance'])

    # Process each group
    for (topic, stance), group in grouped:
        group = group.reset_index(drop=True)  # Reset index for easier indexing

        # List to track which keypoints are to be kept
        keep_indices = set(range(len(group)))

        # Compute pairwise BERTScores for the keypoints in the group
        for i, j in combinations(range(len(group)), 2):  # Pairwise comparisons
            if i not in keep_indices or j not in keep_indices:
                continue  # Skip if one of these indices is already removed

            prediction_1 = preprocess_text(group.loc[i, 'Meltemi_Instruct_4shot'])
            prediction_2 = preprocess_text(group.loc[j, 'Meltemi_Instruct_4shot'])

            # Compute BERTScore
            bertscore_result = bertscore.compute(
                predictions=[prediction_1],
                references=[prediction_2],
                lang='el',  # Change to the appropriate language
                device='cpu'
            )
            bert_f1_score = bertscore_result['f1'][0]

            # If the similarity is too high, remove the lower-ranked keypoint
            if bert_f1_score >= bertscore_threshold:
                if i < j:
                    # Log the dropped keypoint and similarity score
                    dropped_keypoints.append({
                        'topic': topic,
                        'stance': stance,
                        'dropped_keypoint': group.loc[j, 'Meltemi_Instruct_4shot'],
                        'kept_keypoint': group.loc[i, 'Meltemi_Instruct_4shot'],
                        'bertscore': bert_f1_score,
                        'dropped_index': j,
                        'kept_index': i
                    })
                    keep_indices.discard(j)
                else:
                    dropped_keypoints.append({
                        'topic': topic,
                        'stance': stance,
                        'dropped_keypoint': group.loc[i, 'Meltemi_Instruct_4shot'],
                        'kept_keypoint': group.loc[j, 'Meltemi_Instruct_4shot'],
                        'bertscore': bert_f1_score,
                        'dropped_index': i,
                        'kept_index': j
                    })
                    keep_indices.discard(i)

        # Append the remaining rows to the filtered list
        filtered_rows.extend(group.iloc[list(keep_indices)].to_dict('records'))

    # Convert the filtered rows back into a DataFrame
    filtered_df = pd.DataFrame(filtered_rows)

    # Convert the dropped keypoints into a DataFrame
    dropped_keypoints_df = pd.DataFrame(dropped_keypoints)

    return filtered_df, dropped_keypoints_df


In [15]:
# Run the filtering function
filtered_df, dropped_keypoints_df = filter_keypoints_with_logging(df_filtered, bertscore_threshold=0.88)

# Overview of filtered keypoints
print("Filtered Keypoints:\n", filtered_df.head())

# Overview of dropped keypoints
print("Dropped Keypoints Log:\n", dropped_keypoints_df.head())


Filtered Keypoints:
                              Meltemi_Instruct_16shot  \
0  Οι ΗΠΑ αντιμετωπίζουν σοβαρά κοινωνικά και οικ...   
1  Οι Ηνωμένες Πολιτείες έχουν ένα ιστορικό φυλετ...   
2                     Διακρίσεις στην αγορά εργασίας   
3  Η εγκληματικότητα στις ΗΠΑ είναι ένα σημαντικό...   
4  Η υγειονομική περίθαλψη και η εκπαίδευση είναι...   

                                 Representative_Docs  \
0  ['η κατάσταση αγωνίας που βιώνουν πολλές από τ...   
1  ['Οι Ηνωμένες Πολιτείες υπήρξαν για χρόνια κοι...   
2  ['σε ορισμένα μέρη δεν σου δίνεται η ευκαιρία ...   
3  ['σε ορισμένες πολιτείες εξακολουθούν να υπάρχ...   
4  ['υπάρχει πολύ υψηλό ποσοστό ανεργίας και οι υ...   

                                      topic  stance  rouge1_score  \
0  Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς      -1      0.235294   
1  Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς      -1      0.117647   
2  Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς      -1      0.000000   
3  Οι ΗΠΑ είναι μια καλή χώρα

In [16]:
len(filtered_df)

30

In [17]:
dropped_keypoints_df

,topic,stance,dropped_keypoint,kept_keypoint,bertscore,dropped_index,kept_index
0,Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς,1,Οι Ηνωμένες Πολιτείες είναι η πρώτη επιλογή τω...,Οι Ηνωμένες Πολιτείες προσφέρουν ευκαιρίες για...,0.894926,5,3
1,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,-1,Το να είναι υποχρεωτικό το μάθημα σεξουαλικής ...,Η υποχρεωτικότητα των εμβολιασμών παραβιάζει τ...,0.886198,5,1
2,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,1,Τα παιδικά εμβόλια θα πρέπει να είναι υποχρεωτ...,Τα εμβόλια θα πρέπει να είναι υποχρεωτικά για ...,0.946949,1,0
3,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,1,Τα παιδικά εμβόλια ρουτίνας θα πρέπει να είναι...,Τα εμβόλια θα πρέπει να είναι υποχρεωτικά για ...,0.903928,2,0
4,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,1,Ο εμβολιασμός είναι απαραίτητος για την προστα...,Τα εμβόλια θα πρέπει να είναι υποχρεωτικά για ...,0.911997,4,0
5,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,1,Ο εμβολιασμός είναι κρίσιμος για την υγεία των...,Τα εμβόλια θα πρέπει να είναι υποχρεωτικά για ...,0.882281,5,0
6,Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...,1,Ο εμβολιασμός σώζει ζωές,Τα εμβόλια σώζουν ζωές,0.898909,8,3
7,Οι πλατφόρμες κοινωνικής δικτύωσης θα πρέπει ν...,-1,Η ρύθμιση των μέσων κοινωνικής δικτύωσης θα πα...,Η παρέμβαση του κράτους στα μέσα κοινωνικής δι...,0.912464,2,0
8,Οι πλατφόρμες κοινωνικής δικτύωσης θα πρέπει ν...,-1,Η ρύθμιση των μέσων κοινωνικής δικτύωσης θα πα...,Η παρέμβαση του κράτους στα μέσα κοινωνικής δι...,0.938599,5,0
9,Οι πλατφόρμες κοινωνικής δικτύωσης θα πρέπει ν...,1,Η ρύθμιση των οίκων ανοχής είναι απαραίτητη γι...,Η ρύθμιση των μέσων κοινωνικής δικτύωσης είναι...,0.897395,5,0


In [18]:
filtered_df

,Meltemi_Instruct_16shot,Representative_Docs,topic,stance,rouge1_score,f1_bertscore
0,Οι ΗΠΑ αντιμετωπίζουν σοβαρά κοινωνικά και οικ...,['η κατάσταση αγωνίας που βιώνουν πολλές από τ...,Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς,-1,0.235294,0.804285
1,Οι Ηνωμένες Πολιτείες έχουν ένα ιστορικό φυλετ...,['Οι Ηνωμένες Πολιτείες υπήρξαν για χρόνια κοι...,Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς,-1,0.117647,0.814347
2,Διακρίσεις στην αγορά εργασίας,['σε ορισμένα μέρη δεν σου δίνεται η ευκαιρία ...,Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς,-1,0.000000,0.772411
3,Η εγκληματικότητα στις ΗΠΑ είναι ένα σημαντικό...,['σε ορισμένες πολιτείες εξακολουθούν να υπάρχ...,Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς,-1,0.250000,0.773089
4,Η υγειονομική περίθαλψη και η εκπαίδευση είναι...,['υπάρχει πολύ υψηλό ποσοστό ανεργίας και οι υ...,Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς,-1,0.222222,0.777985
5,Το σύστημα υγείας των ΗΠΑ είναι πολύ ακριβό κα...,['στις ΗΠΑ το σύστημα υγείας είναι πολύ ακριβό...,Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς,-1,0.200000,0.817985
6,Οι ΗΠΑ είναι η καλύτερη χώρα για να ζει κανείς...,['Οι ΗΠΑ αντιπροσωπεύουν ένα ευρύ φάσμα πολιτι...,Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς,1,0.592593,0.844676
7,Η Αμερική είναι μια χώρα με πολλές ευκαιρίες γ...,['Προσωπικά πιστεύω ότι είναι μια χώρα που προ...,Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς,1,0.333333,0.837646
8,Οι ΗΠΑ ως παγκόσμια δύναμη ενθαρρύνουν τη μετα...,['Οι ΗΠΑ είναι το αμερικανικό όνειρο γιατί είν...,Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς,1,0.235294,0.809254
9,Οι Ηνωμένες Πολιτείες προσφέρουν ευκαιρίες για...,['στις ΗΠΑ έχεις υγεία και ασφάλεια και μπορεί...,Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς,1,0.125000,0.811463


In [19]:
filtered_df.to_csv('generated_kps_filtered.csv', index=False)